In [ ]:
# Importa= Panda libaries
import pandas as pd
from src.exception import CustomException
from src.logger import logging


## Model Training

In [ ]:
#loading the dateset
df = pd.read_csv('./data/gemstone.csv')
df.head()

In [ ]:
# Droping the column Id
df=df.drop(labels=['id'],axis=1)

In [ ]:
## Independent and dependent features
X = df.drop(labels=['price'],axis=1)
Y = df[['price']]

In [ ]:
X

In [ ]:
Y

In [ ]:
# Segregating numerical and categorical variables
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [ ]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [ ]:
# Importing the libraies
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # Handling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])


In [ ]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=42)

In [ ]:
X_train.head()

In [ ]:
#Creating dataframe of train and test data
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [ ]:
#Model Training
import sklearn.linear_model as linear_model
import sklearn.ensemble as ensemble
from sklearn.tree import DecisionTreeRegressor
import sklearn.svm as svm
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

# List of all regression models
regression_models = [
    linear_model.LinearRegression,
    linear_model.Ridge,
    linear_model.Lasso,
    linear_model.ElasticNet,
    DecisionTreeRegressor,
    ensemble.GradientBoostingRegressor,
    ensemble.RandomForestRegressor,
    svm.SVR,
]

# Print a list of all regression models
print("List of all regression models:")
for model in regression_models:
    print(model.__name__)

In [ ]:
#Applying Linear model
regression=linear_model.LinearRegression()
regression.fit(X_train,y_train)

In [ ]:
#Regresion cofficient
regression.coef_

In [ ]:
#Regression intercept
regression.intercept_

In [ ]:
#Evaluating the  Model creating function
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square


In [19]:
## Train multiple models
## Model Evaluation
models={
    'LinearRegression':linear_model.LinearRegression(),
    'Lasso':linear_model.Lasso(),
    'Ridge':linear_model.Ridge(),
    'Elasticnet':linear_model.ElasticNet(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'GradientBoostingRegressor':ensemble.GradientBoostingRegressor(),
    'RandomForestRegressor':ensemble.RandomForestRegressor(),
    'SVR':svm.SVR(),
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')
    
    
